In [1]:
import torch
from torch import Tensor
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
import numpy as np
import torch as t
import tqdm
#functional
import torch.nn.functional as F
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from functools import partial
import einops
import circuitsvis as cv

# Graphing stuff

In [2]:
def to_numpy(tensor, flat=False):
    if type(tensor)!=torch.Tensor:
        return tensor
    if flat:
        return tensor.flatten().detach().cpu().numpy()
    else:
        return tensor.detach().cpu().numpy()

def imshow(tensor, xaxis=None, yaxis=None, animation_name='Snapshot', **kwargs):
    tensor = torch.squeeze(tensor)
    px.imshow(to_numpy(tensor, flat=False),aspect='auto', 
              labels={'x':xaxis, 'y':yaxis, 'animation_name':animation_name}, 
              **kwargs).show()
# Set default colour scheme
imshow = partial(imshow, color_continuous_scale='Blues')
# Creates good defaults for showing divergent colour scales (ie with both 
# positive and negative values, where 0 is white)
imshow_div = partial(imshow, color_continuous_scale='RdBu', color_continuous_midpoint=0.0)
# Presets a bunch of defaults to imshow to make it suitable for showing heatmaps 
# of activations with x axis being input 1 and y axis being input 2.
inputs_heatmap = partial(imshow, xaxis='Input 1', yaxis='Input 2', color_continuous_scale='RdBu', color_continuous_midpoint=0.0)

def line(x, y=None, hover=None, xaxis='', yaxis='', **kwargs):
    if type(y)==torch.Tensor:
        y = to_numpy(y, flat=True)
    if type(x)==torch.Tensor:
        x = to_numpy(x, flat=True)
    fig = px.line(x, y=y, hover_name=hover, **kwargs)
    fig.update_layout(xaxis_title=xaxis, yaxis_title=yaxis)
    fig.show()

def lines(lines_list, x=None, mode='lines', labels=None, xaxis='', yaxis='', title = '', log_y=False, hover=None, **kwargs):
    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    fig = go.Figure(layout={'title':title})
    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    fig.show()

# Training Config Stuff

In [3]:
cfg = HookedTransformerConfig(
    n_layers = 1,
    n_heads = 1,
    d_model = 128,
    d_head = 128,
    d_mlp = 512,
    act_fn = "relu",
    normalization_type=None,
    d_vocab=11,
    d_vocab_out=10,
    n_ctx=10,
    init_weights=True,
    device="cuda",
    seed = 1337,
)

lr = 1e-3
weight_decay = 1e-4
test_train_split = 0.8
epochs = 100
batch_size = 4096



In [4]:

np_data = np.load('data/moves.npy')

In [5]:
np_data.shape

(46080, 11)

In [6]:
#load npy file
np_data = np.load('data/moves.npy')
data = np_data[:, :-1]
labels = np_data[:, 1:]

print(len(data))
print(len(labels))
print(data[0])
print(labels[0])

46080
46080
[10  0  1  2  3  4  6  5  8  7]
[0 1 2 3 4 6 5 8 7 9]


In [7]:
encoded_labels = F.one_hot(t.tensor(labels))
print(encoded_labels)
print(t.sum(encoded_labels, axis=1))

tensor([[[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 0, 1]],

        [[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 1]],

        [[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 0, 1]],

        ...,

        [[0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 1,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 1]],

        [[0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 1, 0, 0],
         [0,

In [8]:
encoded_data = F.one_hot(t.tensor(data))
print(encoded_data[1238])

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])


In [9]:
encoded_labels[0,:,:]

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [10]:
#data and labels as numpy arrays
data = np.array(data)
encoded_labels = np.array(encoded_labels)
#data and encoded_labels as tensors
data = t.from_numpy(data)
encoded_labels = t.from_numpy(encoded_labels).to(t.float)
total_data = list(zip(data, encoded_labels))
split_data = list(t.utils.data.random_split(total_data, [.8, .2]))
train_pairs = split_data[0]
test_pairs= split_data[1]
train_data, train_labels = zip(*train_pairs)
test_data, test_labels = zip(*test_pairs)

train_data = t.stack(train_data).to(cfg.device)
train_labels = t.stack(train_labels).to(cfg.device)
test_data = t.stack(test_data).to(cfg.device)
test_labels = t.stack(test_labels).to(cfg.device)


#test train split
train_data = data[:int(len(data)*test_train_split)]
train_labels = encoded_labels[:int(len(data)*test_train_split)]
test_data = data[int(len(data)*test_train_split):]
test_labels = encoded_labels[int(len(data)*test_train_split):]

In [11]:
print(len(test_data))
print(len(test_labels))

9216
9216


In [12]:
def loss_fn(logits, labels):
    return t.nn.functional.cross_entropy(logits, labels)

In [13]:
ten = t.tensor([0,1]).to(t.float)
loss_fn(ten, ten)

tensor(0.3133)

In [14]:
train_losses = []
test_losses = []
model = HookedTransformer(cfg).to(cfg.device)
optimizer = t.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

# for epoch in tqdm.tqdm(range(epochs)):
for epoch in range(epochs):
    for batch in range(0, len(train_data), batch_size):
        train_logits = model(train_data[batch:batch+batch_size])
        print(train_logits.dtype)
        print(train_labels.dtype)
        train_loss = loss_fn(train_logits, train_labels[batch:batch+batch_size])

        train_loss.backward()

        train_losses.append(train_loss.item())
        optimizer.step()
        optimizer.zero_grad()

        with t.inference_mode():
            test_logits = model(test_data)
            test_loss = loss_fn(test_logits, test_labels)
            test_losses.append(test_loss.item())

        print(f"Epoch {epoch} | Train Loss: {train_loss.item()} | Test Loss: {test_loss.item()}")

Moving model to device:  cuda


  1%|          | 1/100 [00:00<00:43,  2.28it/s]

torch.float32
torch.float32
Epoch 0 | Train Loss: 2.301990032196045 | Test Loss: 2.2703514099121094
torch.float32
torch.float32
Epoch 0 | Train Loss: 2.271068572998047 | Test Loss: 2.2395405769348145
torch.float32
torch.float32
Epoch 0 | Train Loss: 2.23994779586792 | Test Loss: 2.2085189819335938
torch.float32
torch.float32
Epoch 0 | Train Loss: 2.2092206478118896 | Test Loss: 2.1764636039733887
torch.float32
torch.float32
Epoch 0 | Train Loss: 2.176811695098877 | Test Loss: 2.142791748046875
torch.float32
torch.float32
Epoch 0 | Train Loss: 2.1434266567230225 | Test Loss: 2.10731840133667
torch.float32
torch.float32
Epoch 0 | Train Loss: 2.1076362133026123 | Test Loss: 2.0705783367156982
torch.float32
torch.float32
Epoch 0 | Train Loss: 2.070155382156372 | Test Loss: 2.0336356163024902
torch.float32
torch.float32
Epoch 0 | Train Loss: 2.0340754985809326 | Test Loss: 1.9981554746627808
torch.float32
torch.float32
Epoch 1 | Train Loss: 1.9980801343917847 | Test Loss: 1.965922474861145


  3%|▎         | 3/100 [00:00<00:17,  5.57it/s]

Epoch 2 | Train Loss: 1.7850780487060547 | Test Loss: 1.7634681463241577
torch.float32
torch.float32
Epoch 2 | Train Loss: 1.7640475034713745 | Test Loss: 1.7454668283462524
torch.float32
torch.float32
Epoch 2 | Train Loss: 1.7470600605010986 | Test Loss: 1.7260545492172241
torch.float32
torch.float32
Epoch 2 | Train Loss: 1.7282990217208862 | Test Loss: 1.7056715488433838
torch.float32
torch.float32
Epoch 2 | Train Loss: 1.707759141921997 | Test Loss: 1.6847436428070068
torch.float32
torch.float32
Epoch 2 | Train Loss: 1.684143304824829 | Test Loss: 1.6626578569412231
torch.float32
torch.float32
Epoch 2 | Train Loss: 1.6615384817123413 | Test Loss: 1.63853919506073
torch.float32
torch.float32
Epoch 2 | Train Loss: 1.6412895917892456 | Test Loss: 1.61211359500885
torch.float32
torch.float32
Epoch 3 | Train Loss: 1.615416407585144 | Test Loss: 1.583520531654358
torch.float32
torch.float32
Epoch 3 | Train Loss: 1.589840292930603 | Test Loss: 1.5527479648590088
torch.float32
torch.float32

...

Epoch 94 | Train Loss: 4.9478327127872035e-05 | Test Loss: 4.8700003389967605e-05
torch.float32
torch.float32
Epoch 94 | Train Loss: 4.87815668748226e-05 | Test Loss: 4.859014370595105e-05
torch.float32
torch.float32
Epoch 94 | Train Loss: 4.9275255150860175e-05 | Test Loss: 4.84778756799642e-05
torch.float32
torch.float32
Epoch 94 | Train Loss: 4.844596332986839e-05 | Test Loss: 4.8365298425778747e-05
torch.float32
torch.float32
Epoch 94 | Train Loss: 4.858102693106048e-05 | Test Loss: 4.825624273507856e-05
torch.float32
torch.float32
Epoch 95 | Train Loss: 4.8424219130538404e-05 | Test Loss: 4.814544445252977e-05
torch.float32
torch.float32
Epoch 95 | Train Loss: 4.8499197873752564e-05 | Test Loss: 4.803640331374481e-05
torch.float32
torch.float32
Epoch 95 | Train Loss: 4.7953602916095406e-05 | Test Loss: 4.792683830601163e-05
torch.float32
torch.float32
Epoch 95 | Train Loss: 4.8247802624246106e-05 | Test Loss: 4.781710231327452e-05
torch.float32
torch.float32
Epoch 95 | Train Loss:

 99%|█████████▉| 99/100 [00:09<00:00, 11.12it/s]

Epoch 96 | Train Loss: 4.6502846089424565e-05 | Test Loss: 4.643200009013526e-05
torch.float32
torch.float32
Epoch 96 | Train Loss: 4.663770232582465e-05 | Test Loss: 4.632792479242198e-05
torch.float32
torch.float32
Epoch 97 | Train Loss: 4.648849062505178e-05 | Test Loss: 4.622337291948497e-05
torch.float32
torch.float32
Epoch 97 | Train Loss: 4.656014425563626e-05 | Test Loss: 4.612081102095544e-05
torch.float32
torch.float32
Epoch 97 | Train Loss: 4.603829074767418e-05 | Test Loss: 4.601804539561272e-05
torch.float32
torch.float32
Epoch 97 | Train Loss: 4.6326244046213105e-05 | Test Loss: 4.591475226334296e-05
torch.float32
torch.float32
Epoch 97 | Train Loss: 4.6541401388822123e-05 | Test Loss: 4.5811910240445286e-05
torch.float32
torch.float32
Epoch 97 | Train Loss: 4.588545925798826e-05 | Test Loss: 4.5709151891060174e-05
torch.float32
torch.float32
Epoch 97 | Train Loss: 4.635014192899689e-05 | Test Loss: 4.560881279758178e-05
torch.float32
torch.float32
Epoch 97 | Train Loss: 

100%|██████████| 100/100 [00:09<00:00, 10.79it/s]

Epoch 99 | Train Loss: 4.473371882340871e-05 | Test Loss: 4.431409979588352e-05
torch.float32
torch.float32
Epoch 99 | Train Loss: 4.423230348038487e-05 | Test Loss: 4.421750418259762e-05
torch.float32
torch.float32
Epoch 99 | Train Loss: 4.451301720109768e-05 | Test Loss: 4.411974805407226e-05
torch.float32
torch.float32
Epoch 99 | Train Loss: 4.4723052269546315e-05 | Test Loss: 4.4024262024322525e-05
torch.float32
torch.float32
Epoch 99 | Train Loss: 4.409309622133151e-05 | Test Loss: 4.392766277305782e-05
torch.float32
torch.float32
Epoch 99 | Train Loss: 4.45414443674963e-05 | Test Loss: 4.38314164057374e-05
torch.float32
torch.float32
Epoch 99 | Train Loss: 4.379630263429135e-05 | Test Loss: 4.3735602957895026e-05
torch.float32
torch.float32
Epoch 99 | Train Loss: 4.39273972006049e-05 | Test Loss: 4.364071719464846e-05


In [15]:
seq = t.tensor([10] * 10).to(cfg.device)
out = model(seq)

In [16]:
out[:,-1,:]

tensor([[-68.5691, -53.3248, -33.7109,  28.4673, -47.1521,  49.8403,  20.7787,
          18.6870, -19.6562,  14.9037]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [17]:
line(test_losses, log_y=True)
# plt.plot(train_losses)

In [18]:
line(train_losses, log_y=True)

In [19]:
model

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): Identity()
      (ln2): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_mid): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (unembed): Unembed()
)

In [20]:
imshow(model.embed.W_E)

In [21]:
# Take the dot product of all the embedding vectors
emb = model.embed.W_E
vec_count = emb.shape[0]
vec_dim = emb.shape[1]
print(f"The embedding shape is {emb.shape}, so our vectors of length {emb.shape[1]}")

dot_products = einops.einsum(emb, emb, "v2 embs, v1 emb -> v1 v2")

The embedding shape is torch.Size([11, 128]), so our vectors of length 128


In [22]:
print(dot_products.shape)
imshow_div(dot_products)

torch.Size([11, 11])


## What would your hypothesis around the attention head activations be based on seeing this?
+ Jack - My poorly informed guess is that tokens with low dot products and/or low norms won't have any strong attentional interaction
+ Omar - I think that corner moves [0, 2, 6, 8] will have similar attention patterns
+ Ari - I think same as Omar, plus center attends to everything, middle edges have attention symmetry too

In [23]:
tokens = [0,1,2,3,4,6,5,8,7]
# tokens = ([10] * 5) + [1,2,5,8,7]
str_tokens = [str(token) for token in tokens]
logits, cache = model.run_with_cache(torch.tensor(tokens).to('cuda'), remove_batch_dim=True)


In [24]:
print(type(cache))
attention_pattern = cache["pattern", 0, "attn"]
print(attention_pattern.shape)

<class 'transformer_lens.ActivationCache.ActivationCache'>
torch.Size([1, 10, 10])


In [25]:
cv.attention.attention_patterns(tokens=str_tokens, attention=attention_pattern)